In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/labels.txt
/kaggle/input/dataset/reviews.txt


In [2]:
with open('../input/dataset/reviews.txt','r') as f:
    reviews=f.read()
with open('../input/dataset/labels.txt','r') as f:
    labels=f.read()

In [3]:
print(reviews[:2000])
print(labels[:5])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

# Data preprocessing

In [4]:
from string import punctuation
print(punctuation)
reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])
# print(all_text[:100])
reviews_split = all_text.split('\n')
all_text=''.join(reviews_split)
words=all_text.split()#default separator is the white space
# print(words[:20])


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
print(len(words))

6020196


# Encoding the words

In [6]:
from collections import Counter
## Build a dictionary that maps words to integers
vocab = (Counter(words))
vocab = sorted(vocab, key=vocab.get, reverse=True)

# print(len(vocab))
int_to_vocab = dict(enumerate(vocab))
vocab_to_int = {v:ii+1 for ii,v in int_to_vocab.items()}
# print(vocab_to_int)

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [7]:
# print(reviews_ints[:1])
print(vocab_to_int['the'])

1


In [8]:
#encoding the labels now
labels_split=labels.split('\n')
encoded_labels = np.array([1 if label =='positive' else 0 for label in labels_split])
# print(encoded_labels[10])

In [9]:
#removing outliers
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [10]:
print('Number of reviews before removing outliers:',len(reviews_ints))
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review)!=0]
reviews_ints =[reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii]for ii in non_zero_idx])
print('Number of reviews after removing outliers: ', len(reviews_ints))


Number of reviews before removing outliers: 25001
Number of reviews after removing outliers:  25000


In [11]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints),seq_length),dtype=int)
    for i, row in enumerate(reviews_ints):
        features[i,-len(row):]=np.array(row)[:seq_length]
    return features
        
    

In [12]:
x=np.array([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]])
y=[1,2,3,4]
# print(x[:,-3:],'\n')
# print(x[:,-3])
# print(y[1:4:2])
features = np.zeros((len(x),2),dtype=int)
# print(featu/res[1,1])
z = np.array(x)[:2]
print(z)


[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]


In [13]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
#print(features[:30,:10])


Training, Validation, Test
With our data in nice shape, we'll split it into training, validation, and test sets.

Exercise: Create the training, validation, and test sets.

You'll need to create sets for the features and the labels, train_x and train_y, for example.
Define a split fraction, split_frac as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9.
Whatever data is left will be split in half to create the validation and testing data.

In [14]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx=int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx],features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

#create tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#data loaders
batch_size=50
train_loader = DataLoader(train_data,shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [16]:
sample_x, sample_y = next(iter(train_loader))

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   11,    18,   489,  ...,    47,  1894,   123],
        [    0,     0,     0,  ..., 18536,  1367,    21],
        [    0,     0,     0,  ...,   543,     7,     7],
        ...,
        [  998,  1217,   437,  ...,   312,  5124,    13],
        [    0,     0,     0,  ...,     6,   179,   924],
        [    0,     0,     0,  ...,  1777,     1,   687]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1])


In [17]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [27]:
#Model
import torch.nn as nn
class SentimentRNN(nn.Module):
    def __init__(self,vocab_size, output_size, embedding_dim,hidden_dim,n_layers, drop_prob=0.5):
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        self.embedding_dim =embedding_dim
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,n_layers,
                           dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim,output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        x=x.long()
        embeds = self.embedding(x)
        r_output, hidden = self.lstm(embeds,hidden)
        r_output = r_output[:,-1,:]#geting the last time step output
        out = self.dropout(r_output)
#         out = out.contiguous().view(-1,self.hidden_dim)
        out = self.fc(out)
        sig_out=self.sig(out)
        return sig_out, hidden
    def init_hidden(self, batch_size):
        '''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden
        
        

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(74073, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [30]:
#instantiate the network
vocab_size=len(vocab_to_int)+1
output_size=1
embedding_dim=400
hidden_dim=256
n_layers=2
net=SentimentRNN(vocab_size,output_size,embedding_dim,hidden_dim,n_layers)
lr=0.001
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(net.parameters(),lr=lr)

epochs=4
counter=0
print_every=100
clip=5

if(train_on_gpu):
    net.cuda()

net.train()
for e in range(epochs):
#     net.train()
    h=net.init_hidden(batch_size)
    for inputs,labels in train_loader:
        net.train()
        counter+=1
        if(train_on_gpu):
            inputs,labels = inputs.cuda(), labels.cuda()
        h=tuple([each.data for each in h])
        net.zero_grad()
        output,h = net(inputs,h)
        loss = criterion(output.squeeze(),labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(),clip)
        optimizer.step()
        
        if counter % print_every==0:
            #get validation loss
            val_h=net.init_hidden(batch_size)
            val_losses=[]
            net.eval()
            for inputs,labels in valid_loader:
                if(train_on_gpu):
                    inputs,labels = inputs.cuda(), labels.cuda()
                h=tuple([each.data for each in val_h])
                output,h=net(inputs,h)
                val_loss=criterion(output.squeeze(),labels.float())
                val_losses.append( val_loss.item())
            
            print("Epoch:{}/{}...".format(e+1,epochs),
                       "Step:{}...".format(counter),
                       "Loss:{:.6f}...".format(loss.item()),
                       "Val loss: {:.6f}".format(np.mean(val_losses))
                      )    
         


Epoch:1/4... Step:100... Loss:0.642509... Val loss: 0.655525
Epoch:1/4... Step:200... Loss:0.500495... Val loss: 0.630297
Epoch:1/4... Step:300... Loss:0.611518... Val loss: 0.584168
Epoch:1/4... Step:400... Loss:0.522333... Val loss: 0.543917
Epoch:2/4... Step:500... Loss:0.444236... Val loss: 0.476116
Epoch:2/4... Step:600... Loss:0.384349... Val loss: 0.449689
Epoch:2/4... Step:700... Loss:0.486675... Val loss: 0.464213
Epoch:2/4... Step:800... Loss:0.412699... Val loss: 0.446014
Epoch:3/4... Step:900... Loss:0.503199... Val loss: 0.579040
Epoch:3/4... Step:1000... Loss:0.286742... Val loss: 0.494151
Epoch:3/4... Step:1100... Loss:0.280538... Val loss: 0.413395
Epoch:3/4... Step:1200... Loss:0.178616... Val loss: 0.441001
Epoch:4/4... Step:1300... Loss:0.115510... Val loss: 0.519348
Epoch:4/4... Step:1400... Loss:0.211640... Val loss: 0.459530
Epoch:4/4... Step:1500... Loss:0.149693... Val loss: 0.483193
Epoch:4/4... Step:1600... Loss:0.161172... Val loss: 0.488136


In [31]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.487
Test accuracy: 0.804


In [47]:
# negative test review
test_review_neg = "The best movie I have seen; acting was good and I loved it."
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int.get(word, 0) for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)


[[1, 117, 18, 10, 28, 108, 113, 14, 50, 2, 10, 445, 8]]


In [48]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   1 117  18  10  28 108 113  14  50   2  10
  445   8]]
torch.Size([1, 200])


In [34]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [49]:
predict(net, test_review_neg, seq_length)


Prediction value, pre-rounding: 0.989749
Positive review detected!
